In [1]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import sqlite3
import sys
sys.path.append('../tools') # agregar la ruta de la carpeta tools
from tool import sqlite_tool,handle_tool_call,consulta_filtrada

In [2]:
# Inicialización

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key sin configurar")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


## Prompt de systema

In [3]:
system_message = "Eres un asistente útil para una inmobiliaria de Tunja Boyaca llamada MetroCuadrado. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

## Open Ai

In [11]:
def chat(history):
    messages = [{"role": "system", "content": system_message},{"role":"user","content":"hola como me puedes ayudar"}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=sqlite_tool)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response= handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comenta o elimina la siguiente línea si prefiere omitir el audio por ahora.
    #talker(reply)
    
    return history

In [ ]:
# Código Gradio más complejo, ya que no estamos usando la interfaz de chat predeterminada.
# Si pasamos inbrowser=True en la última línea, aparecerá inmediatamente una ventana Gradio.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image('../image/tunja.png',height=500,interactive=False)
    with gr.Row():
        entry = gr.Textbox(label="Chatea:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


---